# Monte Carlo Simulations for NLLS

In [2]:
from utils import get_batch, fit_biExponential_model_signal, read_data, rRMSE_per_case
from tqdm import tqdm
import numpy as np
import os
import matplotlib.pyplot as plt
from model import PIA
from torch import optim
import torch

In [3]:
file_dir='../public_training_data/'
file_Resultdir='../Result/'
fname_gt ='_IVIMParam.npy'
fname_gtDWI ='_gtDWIs.npy'
fname_tissue ='_TissueType.npy'
fname_noisyDWIk = '_NoisyDWIk.npy'


b_values = [0, 5, 50, 100, 200, 500, 800, 1000]
test_tensor, f_test, D_test, D_star_test, clean = get_batch(10, noise_sdt=0.01)
#test = test_tensor.detach().cpu().numpy()
#f, D, D_star = fit_biExponential_model_signal(test, b_values)

In [44]:
from torch import nn
from torch.nn import functional as F


class PIA(nn.Module):
    def __init__(self, b_values = [0, 5, 50, 100, 200, 500, 800, 1000],
                hidden_dims= [32, 64, 128, 256, 512],
                predictor_depth=2):
         super(PIA, self).__init__()

         self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
         self.number_of_signals = len(b_values)
         self.sigmoid = nn.Sigmoid()
         modules = []
         in_channels = self.number_of_signals
         for h_dim in hidden_dims:
             modules.append(nn.Sequential(
                 nn.Linear(in_features=in_channels, out_features=h_dim),
                 nn.LeakyReLU()))
             in_channels = h_dim
         self.encoder = nn.Sequential(*modules).to(self.device)
         D_predictor = []
         for _ in range(predictor_depth):
             D_predictor.append(nn.Sequential(
                 nn.Linear(in_features=hidden_dims[-1], out_features=hidden_dims[-1]),
                 nn.LeakyReLU())
                 )
         D_predictor.append(nn.Linear(hidden_dims[-1], 1))
         self.D_predictor = nn.Sequential(*D_predictor).to(self.device)
    def encode(self, x):
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(x)   
        D = self.D_predictor(result)
        # D_star = 3 + self.relu(self.D_star_predictor(result))
        # f = self.sigmoid(self.f_predictor(result))      
        return D#[f, D, D_star]

# D_star_predictor = []
# for _ in range(predictor_depth):
    
#     D_star_predictor.append(
#         nn.Sequential(
#             nn.Linear(in_features=hidden_dims[-1], out_features=hidden_dims[-1]),
#             nn.LeakyReLU())
#     )
# D_star_predictor.append(nn.Linear(hidden_dims[-1], 1))
# D_star_predictor = nn.Sequential(*D_star_predictor).to(device)
# f_predictor = []
# for _ in range(predictor_depth):
    
#     f_predictor.append(
#         nn.Sequential(
#             nn.Linear(in_features=hidden_dims[-1], out_features=hidden_dims[-1]),
#             nn.LeakyReLU())
#     )
# f_predictor.append(nn.Linear(hidden_dims[-1], 1))
# f_predictor = nn.Sequential(*f_predictor).to(device)



In [71]:
model = PIA()
params = model.parameters()
lr = 3e-4
optimizer = optim.Adam(params, lr=lr)

In [72]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
test_tensor = test_tensor.to(device)

In [73]:
clean = clean.to(device)
f_test = f_test.to(device)
D_test = D_test.to(device)
loss_fcn = nn.MSELoss()
model.train()
for ep in range(500):
    for i in range(10):
        D = model.encode(test_tensor[i])
        #D_star = 5 + D_star_predictor(result)
        #f = sigmoid(f_predictor(result))
        #signal = torch.zeros((D.shape[0], number_of_signals)).to(device)
        # D_T, D_star_T, f_T = D.T, D_star.T, f.T
        # for inx, b in enumerate(b_values):
        #     signal[:, inx] = (1-f_T)*torch.exp(-(b/1000)*D_T) + f_T*torch.exp(-(b/1000)*D_star_T)
    
        loss = torch.mean((D - D_test[i]) ** 2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if not ep%10:
        print(f'{loss.item():.4f}')





0.0153
0.0613
0.0069
0.0073
0.0069
0.0070
0.0070
0.0067
0.0068
0.0066
0.0065
0.0062
0.0054
0.0036
0.0013
0.0000
0.0016
0.0006
0.0000
0.0002
0.0000
0.0015
0.0021
0.0002
0.0000
0.0057
0.0000
0.0003
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000
0.0000


In [74]:
D = model.encode(test_tensor)

In [75]:
D

tensor([[0.8995],
        [1.4721],
        [0.3933],
        [2.3882],
        [2.4022],
        [0.2531],
        [0.7275],
        [1.9818],
        [1.3470],
        [0.1006]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [70]:
D_test

tensor([0.8995, 1.4721, 0.3933, 2.3882, 2.4022, 0.2531, 0.7275, 1.9818, 1.3470,
        0.1006], device='cuda:0')

In [3]:
dev = "cuda:0"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
device = torch.device(dev)
#models =  []
torch.cuda.empty_cache()
model = PIA(predictor_depth=2).float().to(device).train()
params = model.parameters()
lr = 3e-4
optimizer = optim.Adam(params, lr=lr)
test_tensor = test_tensor.cuda()

clean = clean.cuda()

In [4]:
for ep in tqdm(range(50)):
    optimizer.zero_grad()
    f, D, D_star = model.encode(test_tensor)  
    recon = model.decode(f, D, D_star).cuda()
    loss = model.loss_function(recon, clean)
    loss.backward()
    optimizer.step()
    print(f, f_test)


  2%|▏         | 1/50 [00:00<00:13,  3.70it/s]

tensor([[0.4933]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])


  4%|▍         | 2/50 [00:00<00:09,  4.98it/s]

tensor([[0.4989]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5046]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])


  8%|▊         | 4/50 [00:00<00:09,  5.02it/s]

...

tensor([[0.5463]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5440]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])


 88%|████████▊ | 44/50 [00:04<00:00,  9.74it/s]

tensor([[0.5416]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5394]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5372]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5352]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5335]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])
tensor([[0.5320]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])


100%|██████████| 50/50 [00:04<00:00, 10.92it/s]

tensor([[0.5306]], device='cuda:0', grad_fn=<SigmoidBackward0>) tensor([0.2338])


In [10]:
f_test

tensor([0.4810])

In [ ]:
for image_number in range(3):
    nlls = np.load(os.path.join(file_Resultdir, f'{(image_number + 1):04d}.npy'))
    params = read_data(file_dir, fname_gt, image_number + 1)
fig, ax = plt.subplots(3,1, figsize=(15,45))
title = ['f ', 'D ', 'D_star']
ylims = [(0.0, 1.0), (0.0, 0.003), (0.003, 0.3)]
for c in range(3):
    y = nlls[:, :, c].flatten() 
    x = params[:, :, c].flatten()
    #nbins=30
    #k = gaussian_kde([x,y])
    #xi, yi = np.mgrid[x.min():x.max():nbins*1j, y.min():y.max():nbins*1j]
    #zi = k(np.vstack([xi.flatten(), yi.flatten()]))
    #ax[c].pcolormesh(xi, yi, zi.reshape(xi.shape), cmap="hot", shading='auto')
    ax[c].scatter(x,y, color='b', s=4, alpha=0.5)

    err = np.mean(np.abs(x - y))
    corr = np.corrcoef(x,y)[0,1]
    ax[c].xaxis.set_tick_params(labelsize=20)
    ax[c].yaxis.set_tick_params(labelsize=20)
    #ax[r,c].set_title(fr'{title[c]}, MAE = {err:.3f}, $\rho$ = {corr:.3f}')
    ax[c].set_xlabel('true', fontsize=24)
    ax[c].set_ylabel('predicted', fontsize=24)



#     tissue = read_data(file_dir, fname_tissue, image_number + 1)
#     print(rRMSE_per_case(nlls[:,:,0],nlls[:,:,1],nlls[:,:,2],params[:,:,0],params[:,:,1],params[:,:,2],tissue))
    #k = read_data(file_dir, fname_noisyDWIk, image_number)
    #
    #clean = read_data(file_dir, fname_gtDWI, image_number)
    #clean_real = np.real(clean[:, :, 0])
    #noisy = np.abs(np.fft.ifft2(k, axes=(0,1) ,norm='ortho'))
    #coordBody = np.argwhere(clean > 0)
    #pure_noise_re[b_value].append(noisy_real[coordBody[:,0], coordBody[:,1]] - clean_real[coordBody[:,0], coordBody[:,1]])
    #f_measurement = params[:, :, 0]
    #D_measurement = params[:, :, 1]
    #D_star_measurement = params[:, :, 2]
    #
    #f.append(f_measurement[coordBody[:,0], coordBody[:,1]])
    #D.append(D_measurement[coordBody[:,0], coordBody[:,1]])
    #D_star.append(D_star_measurement[coordBody[:,0], coordBody[:,1]])


# f = np.concatenate(f).ravel()
# a = plt.hist(f, bins=100)
# plt.title('f')

# plt.figure()
# D = np.concatenate(D).ravel()
# a = plt.hist(D, bins=100)
# plt.title('D')

# plt.figure()
# D_star = np.concatenate(D_star).ravel()
# a = plt.hist(D_star, bins=100)
# plt.title('D_star')